Following the tutorials on the <a href="https://hmmlearn.readthedocs.io/en/stable/#user-guide-table-of-contents">hmmlearn docs</a>

# Background
The HMM is a generative probabilistic model. Under this framework, we have a sequence of _observable_ $X$ variables generated by a sequence of _internal hidden states_ $Z$. As the name implies, hidden states are not observed directly. 

_Transitions_ between hidden states are assumed to have the form of a (first-order) Markov chain. The transitions can be specified by a start probability vector $\pi$ and an unchanging transition probability matrix $A$.

The _emission probability_ of an observable $x_t$ can be any distribution with parameters $\theta$ conditioned on the current hidden state $z_t$.

An HMM is fully specified by $\pi$, $A$, and $\theta$.

There are three fundamental problems for HMMs:
1. Given the model parameters and observed data, estimate the optimal sequence of hidden states.
2. Given the model parameters and observed data, calculate the model likelihood.
3. Given just the observed data, estimate the model parameters.